<a href="https://colab.research.google.com/github/metamatematico/Experimentos-IA/blob/master/Guardar_modelos_y_cargarlos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Conectar nuestro google drive

Seguir el link  para obtener el código que conecta nuestra notebook con colab

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


## Se instalan las librerias adecuadas

In [ ]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 1.8MB 6.7MB/s 
     |████████████████████████████████| 890kB 33.3MB/s 
     |████████████████████████████████| 3.2MB 49.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=4383a1324271373c757dd8576bc81cd4d62692b09201ae203dc66b2be70bab22
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 163kB 8.3MB/s 
     |████████████████████████████████| 20.7MB 1.4MB/s 
     |████████████████████████████████| 245kB 26.1MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


## Opción uno: el modelo se hacer con código de python

Aquí se carga la librería y se hacer lo que se tenga que hacer para actualizar el modelo

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer, BertTokenizer, BertForSequenceClassification, AutoModelForSequenceClassification, BertModel, AutoConfig
import numpy as np

MODEL_NAME = "skimai/electra-small-spanish" # Un modelo más chiquito
model = AutoModel.from_pretrained(MODEL_NAME,output_hidden_states=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Aquí tendría que ir más código para actualizar el código

### Guardar modelo

In [ ]:
# Este es la variable con el modelo que queremos guardar
model_to_save = model

# Estos son ejemplso de paths # Segurarse de que existan los directorios
output_dir = '/content/gdrive/My Drive/models/electra-small-spanish'
output_model_file = output_dir+'mimodelo.bin'
!mkdir -p '{output_dir}'


torch.save(model_to_save, output_model_file)
tokenizer.save_pretrained(output_dir)
config_file = AutoConfig.from_pretrained(MODEL_NAME)
config_file.save_pretrained(output_dir)

print('All files saved')


All files saved


### Mostrar modelo

In [ ]:
!ls '{output_dir}'

config.json  special_tokens_map.json  tokenizer_config.json  vocab.txt


## Opción dos: El modelo es creado por un script en la máquina virtual de colab

El modelo es creado un script externo

In [ ]:
# Bajando el script y los datos 
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/language-modeling/run_clm.py
!wget https://gist.githubusercontent.com/jsdario/6d6c69398cb0c73111e49f1218960f79/raw/8d4fc4548d437e2a7203a5aeeace5477f598827d/el_quijote.txt

--2021-02-11 22:19:51--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/language-modeling/run_clm.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17809 (17K) [text/plain]
Saving to: ‘run_clm.py’

run_clm.py          100%[===================>]  17.39K  --.-KB/s    in 0.001s  

2021-02-11 22:19:52 (25.9 MB/s) - ‘run_clm.py’ saved [17809/17809]

--2021-02-11 22:19:52--  https://gist.githubusercontent.com/jsdario/6d6c69398cb0c73111e49f1218960f79/raw/8d4fc4548d437e2a7203a5aeeace5477f598827d/el_quijote.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.110.133|:443... connected.
HT

In [ ]:
!python run_clm.py \
    --model_name_or_path 'datificate/gpt2-small-spanish' \
    --train_file 'el_quijote.txt' \
    --do_train \
    --block_size 216\
    --num_train_epochs 1 \
    --output_dir /output/gpt2-small-spanish-quijote

2021-02-11 22:40:37.176640: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Traceback (most recent call last):
  File "run_clm.py", line 416, in <module>
    main()
  File "run_clm.py", line 162, in main
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()
  File "/usr/local/lib/python3.6/dist-packages/transformers/hf_argparser.py", line 180, in parse_args_into_dataclasses
    obj = dtype(**inputs)
  File "<string>", line 10, in __init__
  File "run_clm.py", line 145, in __post_init__
    assert extension in ["csv", "json", "txt"], "`train_file` should be a csv, a json or a txt file."
AssertionError: `train_file` should be a csv, a json or a txt file.


In [ ]:
# El modelo en la máquina virtal
!ls '/output/gpt2-small-spanish-quijote'
# Moviendo el modelo al google drive
output_dir = '/content/gdrive/My Drive/models/'
!cp -r '/output/gpt2-small-spanish-quijote' '{output_dir}'
!ls '{output_dir}/gpt2-small-spanish-quijote'

config.json	   special_tokens_map.json  training_args.bin
merges.txt	   tokenizer_config.json    train_results.txt
pytorch_model.bin  trainer_state.json	    vocab.json
config.json	   special_tokens_map.json  training_args.bin
merges.txt	   tokenizer_config.json    train_results.txt
pytorch_model.bin  trainer_state.json	    vocab.json


## Recuperando el modelo

In [ ]:
from transformers import pipeline
model = "/content/gdrive/My Drive/models/gpt2-small-spanish-quijote"
model_text = pipeline('text-generation',model=model, tokenizer=model,config={'max_length':5})

In [ ]:
print(model_text("estaba un niño")[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


estaba un niño en la parte de un campo muerto en una larga batalla. Los cuatro de nosotros se pusieron a buscar a aquel cielo que se había visto sobre el arroyo de su padre, porque en estos tiempos, y al estar en
